# Llama 3를 이용해서 텍스트 요약(Text Summarization) 하기
## 작성자 : AISchool ( http://aischool.ai/%ec%98%a8%eb%9d%bc%ec%9d%b8-%ea%b0%95%ec%9d%98-%ec%b9%b4%ed%85%8c%ea%b3%a0%eb%a6%ac/ )
## Reference : https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct
## 논문자료요약 데이터 : https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=90

# 라이브러리 설치

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

## 논문자료요약 데이터 불러오기

In [ ]:
# 2020-02-018.논문자료요약데이터/라벨링데이터/sample_논문/논문요약_0224_0.json

In [ ]:
import json

# 파일 열기 및 JSON 데이터 읽기
with open('논문요약_0224_0.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [ ]:
data

In [ ]:
print('전체 데이터 개수 :', data['totalcount'])

전체 데이터 개수 : 11802


In [ ]:
abstract_data = data['data']
abstract_data

In [ ]:
abstract_data[0]

{'doc_type': '논문',
 'doc_id': 'A201007211374',
 'title': '국내 검사실에서의 대변 검경 검사에 대한 내부정도관리 현황',
 'date': '2018',
 'reg_no': 'ART002294415',
 'ipc': '의약학',
 'issued_by': '대한진단검사의학회',
 'author': '권용준 (제1)|||원은정 (교신)|||기승정 (참여)|||김수현 (참여)|||신명근 (참여)|||신종희 (참여)|||서순팔 (참여)',
 'summary_entire': [{'orginal_text': '배경: 현재 대변검경검사에서는 적절한 정도관리물질이 없어 질관리가 미진한 실정이다. 본 연구의 목적은 설사변에서 발견되는 장내 기생충 현황을 알아보고, 대변 검경에 있어 활용 가능한 정도관리물질을 제조하여 그 안정성을 검증하고자 하였다. 방법: 본 연구는 대변배양이 의뢰된 검체 중 82건의 설사변을 대상으로 하였다. 현미경적 진단법은 직접도말법과 포르말린-에테르침전법으로 시행하였다. 작은와포자충, 람블편모충, 이질아메바에대해서는 ELISA 키트 (R-Biopharm) 및 xTAG® Gastrointestinal Pathogen Panel (Luminex Corporation) 을 이용하여 검사하였다. 이핵아메바와 블라스토시스티스 호미니스에 대해서는 특이 유전자를 타깃으로 하는 PCR을 수행하였다. 정도관리물질은 음성 대변 검체와 동해긴촌충의 충란을 섞어 제조하였다. 제조된 정도관리물질은 보존제 종류, 보관 온도, 보관 기간에 따라 안정성을 평가하였다. 결과: 총 82건의 설사변에서 현미경법으로는 유의한 기생충을 발견할 수 없었다. 작은와포자충, 람블편모충, 이질아메바에 대한 ELISA 결과, 82건 모두 음성이었다. xTAG® Gastrointestinal Pathogen Panel을 이용한 결과, 작은와포자충 2건과 람블편모충 1건의양성 소견을 보였다. 이핵아메바와 블라스토시스티스 호미니스 유전체에 대한 PCR 검사 결과

In [ ]:
abstract_data[0].keys()

dict_keys(['doc_type', 'doc_id', 'title', 'date', 'reg_no', 'ipc', 'issued_by', 'author', 'summary_entire', 'summary_section'])

In [ ]:
# summary_entire 내의 orginal_text와 summary_text 추출
extracted_summaries = [{
    'orginal_text': entry['orginal_text'],
    'summary_text': entry['summary_text']
} for item in abstract_data for entry in item['summary_entire']]

In [ ]:
extracted_summaries

[{'orginal_text': '배경: 현재 대변검경검사에서는 적절한 정도관리물질이 없어 질관리가 미진한 실정이다. 본 연구의 목적은 설사변에서 발견되는 장내 기생충 현황을 알아보고, 대변 검경에 있어 활용 가능한 정도관리물질을 제조하여 그 안정성을 검증하고자 하였다. 방법: 본 연구는 대변배양이 의뢰된 검체 중 82건의 설사변을 대상으로 하였다. 현미경적 진단법은 직접도말법과 포르말린-에테르침전법으로 시행하였다. 작은와포자충, 람블편모충, 이질아메바에대해서는 ELISA 키트 (R-Biopharm) 및 xTAG® Gastrointestinal Pathogen Panel (Luminex Corporation) 을 이용하여 검사하였다. 이핵아메바와 블라스토시스티스 호미니스에 대해서는 특이 유전자를 타깃으로 하는 PCR을 수행하였다. 정도관리물질은 음성 대변 검체와 동해긴촌충의 충란을 섞어 제조하였다. 제조된 정도관리물질은 보존제 종류, 보관 온도, 보관 기간에 따라 안정성을 평가하였다. 결과: 총 82건의 설사변에서 현미경법으로는 유의한 기생충을 발견할 수 없었다. 작은와포자충, 람블편모충, 이질아메바에 대한 ELISA 결과, 82건 모두 음성이었다. xTAG® Gastrointestinal Pathogen Panel을 이용한 결과, 작은와포자충 2건과 람블편모충 1건의양성 소견을 보였다. 이핵아메바와 블라스토시스티스 호미니스 유전체에 대한 PCR 검사 결과, 이핵아메바 양성 10건, 블라스토시스티스 호미니스 양성 2건이 관찰되었다. 본 연구에서 제조한 고농도정도관리물질은 3개월 보관 시까지 2가지 보존제 및 보관 온도와무관하게 충란이 관찰되었다. 저농도 정도관리물질의 경우, 14일보관 시까지는 2가지 보존제 및 보관 온도와 무관하게 충란이 관찰되었지만, 3주 이후에는 관찰되지 않았다. 결론: 대변 검경용 정도관리물질의 제조에 있어서 국내 설사환자에서 주로 발견되는 이핵아메바와 블라스토시스티스 호미니스가 포함되는 것이 필요하고, 저농도 물질의 경우 안정성이 떨어

In [ ]:
len(extracted_summaries)

11802

In [ ]:
extracted_summaries[0]

{'orginal_text': '배경: 현재 대변검경검사에서는 적절한 정도관리물질이 없어 질관리가 미진한 실정이다. 본 연구의 목적은 설사변에서 발견되는 장내 기생충 현황을 알아보고, 대변 검경에 있어 활용 가능한 정도관리물질을 제조하여 그 안정성을 검증하고자 하였다. 방법: 본 연구는 대변배양이 의뢰된 검체 중 82건의 설사변을 대상으로 하였다. 현미경적 진단법은 직접도말법과 포르말린-에테르침전법으로 시행하였다. 작은와포자충, 람블편모충, 이질아메바에대해서는 ELISA 키트 (R-Biopharm) 및 xTAG® Gastrointestinal Pathogen Panel (Luminex Corporation) 을 이용하여 검사하였다. 이핵아메바와 블라스토시스티스 호미니스에 대해서는 특이 유전자를 타깃으로 하는 PCR을 수행하였다. 정도관리물질은 음성 대변 검체와 동해긴촌충의 충란을 섞어 제조하였다. 제조된 정도관리물질은 보존제 종류, 보관 온도, 보관 기간에 따라 안정성을 평가하였다. 결과: 총 82건의 설사변에서 현미경법으로는 유의한 기생충을 발견할 수 없었다. 작은와포자충, 람블편모충, 이질아메바에 대한 ELISA 결과, 82건 모두 음성이었다. xTAG® Gastrointestinal Pathogen Panel을 이용한 결과, 작은와포자충 2건과 람블편모충 1건의양성 소견을 보였다. 이핵아메바와 블라스토시스티스 호미니스 유전체에 대한 PCR 검사 결과, 이핵아메바 양성 10건, 블라스토시스티스 호미니스 양성 2건이 관찰되었다. 본 연구에서 제조한 고농도정도관리물질은 3개월 보관 시까지 2가지 보존제 및 보관 온도와무관하게 충란이 관찰되었다. 저농도 정도관리물질의 경우, 14일보관 시까지는 2가지 보존제 및 보관 온도와 무관하게 충란이 관찰되었지만, 3주 이후에는 관찰되지 않았다. 결론: 대변 검경용 정도관리물질의 제조에 있어서 국내 설사환자에서 주로 발견되는 이핵아메바와 블라스토시스티스 호미니스가 포함되는 것이 필요하고, 저농도 물질의 경우 안정성이 떨어질

# Llama 3 모델 불러오기

In [ ]:
import os
os.environ['HF_TOKEN']="여러분의 HuggingFace Access Token값"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
def generate_response(system_message, user_message):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )
    response = outputs[0][input_ids.shape[-1]:]

    return tokenizer.decode(response, skip_special_tokens=True)

In [ ]:
print(model.device)

cuda:0


# Llama 3의 텍스트 요약 성능을 정성적으로 측정해보기

In [ ]:
extracted_summaries[0]

{'orginal_text': '배경: 현재 대변검경검사에서는 적절한 정도관리물질이 없어 질관리가 미진한 실정이다. 본 연구의 목적은 설사변에서 발견되는 장내 기생충 현황을 알아보고, 대변 검경에 있어 활용 가능한 정도관리물질을 제조하여 그 안정성을 검증하고자 하였다. 방법: 본 연구는 대변배양이 의뢰된 검체 중 82건의 설사변을 대상으로 하였다. 현미경적 진단법은 직접도말법과 포르말린-에테르침전법으로 시행하였다. 작은와포자충, 람블편모충, 이질아메바에대해서는 ELISA 키트 (R-Biopharm) 및 xTAG® Gastrointestinal Pathogen Panel (Luminex Corporation) 을 이용하여 검사하였다. 이핵아메바와 블라스토시스티스 호미니스에 대해서는 특이 유전자를 타깃으로 하는 PCR을 수행하였다. 정도관리물질은 음성 대변 검체와 동해긴촌충의 충란을 섞어 제조하였다. 제조된 정도관리물질은 보존제 종류, 보관 온도, 보관 기간에 따라 안정성을 평가하였다. 결과: 총 82건의 설사변에서 현미경법으로는 유의한 기생충을 발견할 수 없었다. 작은와포자충, 람블편모충, 이질아메바에 대한 ELISA 결과, 82건 모두 음성이었다. xTAG® Gastrointestinal Pathogen Panel을 이용한 결과, 작은와포자충 2건과 람블편모충 1건의양성 소견을 보였다. 이핵아메바와 블라스토시스티스 호미니스 유전체에 대한 PCR 검사 결과, 이핵아메바 양성 10건, 블라스토시스티스 호미니스 양성 2건이 관찰되었다. 본 연구에서 제조한 고농도정도관리물질은 3개월 보관 시까지 2가지 보존제 및 보관 온도와무관하게 충란이 관찰되었다. 저농도 정도관리물질의 경우, 14일보관 시까지는 2가지 보존제 및 보관 온도와 무관하게 충란이 관찰되었지만, 3주 이후에는 관찰되지 않았다. 결론: 대변 검경용 정도관리물질의 제조에 있어서 국내 설사환자에서 주로 발견되는 이핵아메바와 블라스토시스티스 호미니스가 포함되는 것이 필요하고, 저농도 물질의 경우 안정성이 떨어질

In [ ]:
orginal_text = extracted_summaries[0]['orginal_text']
summary_text = extracted_summaries[0]['summary_text']
print(orginal_text)
print(summary_text)

배경: 현재 대변검경검사에서는 적절한 정도관리물질이 없어 질관리가 미진한 실정이다. 본 연구의 목적은 설사변에서 발견되는 장내 기생충 현황을 알아보고, 대변 검경에 있어 활용 가능한 정도관리물질을 제조하여 그 안정성을 검증하고자 하였다. 방법: 본 연구는 대변배양이 의뢰된 검체 중 82건의 설사변을 대상으로 하였다. 현미경적 진단법은 직접도말법과 포르말린-에테르침전법으로 시행하였다. 작은와포자충, 람블편모충, 이질아메바에대해서는 ELISA 키트 (R-Biopharm) 및 xTAG® Gastrointestinal Pathogen Panel (Luminex Corporation) 을 이용하여 검사하였다. 이핵아메바와 블라스토시스티스 호미니스에 대해서는 특이 유전자를 타깃으로 하는 PCR을 수행하였다. 정도관리물질은 음성 대변 검체와 동해긴촌충의 충란을 섞어 제조하였다. 제조된 정도관리물질은 보존제 종류, 보관 온도, 보관 기간에 따라 안정성을 평가하였다. 결과: 총 82건의 설사변에서 현미경법으로는 유의한 기생충을 발견할 수 없었다. 작은와포자충, 람블편모충, 이질아메바에 대한 ELISA 결과, 82건 모두 음성이었다. xTAG® Gastrointestinal Pathogen Panel을 이용한 결과, 작은와포자충 2건과 람블편모충 1건의양성 소견을 보였다. 이핵아메바와 블라스토시스티스 호미니스 유전체에 대한 PCR 검사 결과, 이핵아메바 양성 10건, 블라스토시스티스 호미니스 양성 2건이 관찰되었다. 본 연구에서 제조한 고농도정도관리물질은 3개월 보관 시까지 2가지 보존제 및 보관 온도와무관하게 충란이 관찰되었다. 저농도 정도관리물질의 경우, 14일보관 시까지는 2가지 보존제 및 보관 온도와 무관하게 충란이 관찰되었지만, 3주 이후에는 관찰되지 않았다. 결론: 대변 검경용 정도관리물질의 제조에 있어서 국내 설사환자에서 주로 발견되는 이핵아메바와 블라스토시스티스 호미니스가 포함되는 것이 필요하고, 저농도 물질의 경우 안정성이 떨어질 수있음을 감안하여야 할 것이다.

In [ ]:
llama3_summary_text = generate_response(system_message="너는 요약을 수행하는 챗봇이야. 핵심 내용만 256 토큰 이내로 한국어로 요약해줘",
                             user_message=orginal_text)
print(llama3_summary_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


한국어 요약 (256 토큰 이내):

대변검경에서 적절한 정도관리물질이 없는 문제가 있다. 본 연구에서는 설사변에서 발견되는 장내 기생충 현황을 조사하고, 정도관리물질을 제조하여 안정성을 검증하였다. 82건의 설사변을 대상으로 하여 현미경적 진단법, ELISA, xTAG, PCR 등을 사용하여 기생충을 검사하였다. 결과적으로, 이핵아메바와 블라스토시스티스 호미니스가 주로 발견되었으며, 정도관리물질을 제조하여 3개월 동안 안정성을 확인하였다. 하지만, 저농도 정도관리물질의 경우 안정성이 떨어질 수 있음을 감안하여야 할 것이다.


In [ ]:
import textwrap

def print_texts_nicely(orginal_text, summary_text, llama3_summary_text, width=80):
    """
    입력된 세 개의 텍스트를 보기 좋게 출력하는 함수입니다. 긴 텍스트는 주어진 너비에 맞춰 자동으로 줄바꿈을 적용합니다.

    Parameters:
    orginal_text (str): 원본 텍스트
    summary_text (str): 요약 텍스트
    llama3_summary_text (str): Llama3 요약 텍스트
    width (int): 텍스트를 줄바꿈할 때 사용할 최대 문자 너비
    """
    wrapper = textwrap.TextWrapper(width=width)  # 지정된 너비에 맞춰 텍스트를 줄바꿈하기 위한 wrapper 객체 생성

    print("Original Text:")
    print(wrapper.fill(orginal_text))  # wrapper.fill 함수를 사용하여 텍스트 줄바꿈 적용
    print("\n")

    print("Summary Text:")
    print(wrapper.fill(summary_text))
    print("\n")

    print("Llama3 Summary Text:")
    print(wrapper.fill(llama3_summary_text))
    print("\n")

In [ ]:
print_texts_nicely(orginal_text, summary_text, llama3_summary_text)

Original Text:
배경: 현재 대변검경검사에서는 적절한 정도관리물질이 없어 질관리가 미진한 실정이다. 본 연구의 목적은 설사변에서 발견되는 장내 기생충 현황을
알아보고, 대변 검경에 있어 활용 가능한 정도관리물질을 제조하여 그 안정성을 검증하고자 하였다. 방법: 본 연구는 대변배양이 의뢰된 검체 중
82건의 설사변을 대상으로 하였다. 현미경적 진단법은 직접도말법과 포르말린-에테르침전법으로 시행하였다. 작은와포자충, 람블편모충,
이질아메바에대해서는 ELISA 키트 (R-Biopharm) 및 xTAG® Gastrointestinal Pathogen Panel
(Luminex Corporation) 을 이용하여 검사하였다. 이핵아메바와 블라스토시스티스 호미니스에 대해서는 특이 유전자를 타깃으로 하는
PCR을 수행하였다. 정도관리물질은 음성 대변 검체와 동해긴촌충의 충란을 섞어 제조하였다. 제조된 정도관리물질은 보존제 종류, 보관 온도, 보관
기간에 따라 안정성을 평가하였다. 결과: 총 82건의 설사변에서 현미경법으로는 유의한 기생충을 발견할 수 없었다. 작은와포자충, 람블편모충,
이질아메바에 대한 ELISA 결과, 82건 모두 음성이었다. xTAG® Gastrointestinal Pathogen Panel을 이용한 결과,
작은와포자충 2건과 람블편모충 1건의양성 소견을 보였다. 이핵아메바와 블라스토시스티스 호미니스 유전체에 대한 PCR 검사 결과, 이핵아메바 양성
10건, 블라스토시스티스 호미니스 양성 2건이 관찰되었다. 본 연구에서 제조한 고농도정도관리물질은 3개월 보관 시까지 2가지 보존제 및 보관
온도와무관하게 충란이 관찰되었다. 저농도 정도관리물질의 경우, 14일보관 시까지는 2가지 보존제 및 보관 온도와 무관하게 충란이 관찰되었지만,
3주 이후에는 관찰되지 않았다. 결론: 대변 검경용 정도관리물질의 제조에 있어서 국내 설사환자에서 주로 발견되는 이핵아메바와 블라스토시스티스
호미니스가 포함되는 것이 필요하고, 저농도 물질의 경우 안정성이 떨어질 수있

In [ ]:
extracted_summaries[9]

{'orginal_text': '본 연구는 SNS의 이용과 우울의 관계에서 인지적 유연성에 의해 조절된 상향비교의 매개효과를 인스타그램을 중심으로 탐색하였다. 이를 위해 대학생 186명을 대상으로 인스타그램 이용강도, 인스타그램 이용 시 상향비교 경험, 인지적 유연성, 우울을 측정하는 척도에 응답하도록 하였다. 분석 결과 인스타그램 이용강도는 상향비교 및 우울과 정적 상관이 있었고 인지적 유연성과 부적 상관을 보였다. 전체 참가자를 대상으로 인스타그램 이용강도와 우울의 관계에서 상향비교의 매개효과를 검증한 결과 상향비교는 부분매개효과가 있는 것으로 밝혀졌으며, 매개변인인 상향비교와 종속변인인 우울의 관계를 인지적 유연성의 하위요인인 통제유연성이 조절하는 것으로 나타났다. 마지막으로 SPSS Macro를 이용하여 인스타그램 이용강도가 우울에 영향을 미침에 있어서 통제유연성에 의해 조절된 상향비교의 매개효과를 확인하였다. 이러한 결과는 인스타그램 이용강도가 강할수록 인스타그램 공간에서 상향비교를 더욱 많이 경험하게 되어 우울로 이어질 수 있지만 통제유연성을 높일 수 있다면 상향비교의 간접효과가 낮아져서 위험을 완화시킬 수 있음을 의미한다. 마지막으로 본 연구의 의의와 한계점 및 추후 연구에 대해 논의하였다.',
 'summary_text': '본 연구는 인스타그램을 중심으로 SNS의 이용과 우울의 관계에 상향비교가 어떠한 매개를 하는지  탐색하였다. 분석 결과 인스타그램 이용강도가 강할수록 상향비교를 하게 되어 우울로 이어질 수 있지만 통제유연성을 높이면 상향비교를 낮출 수 있어 우울증 위험을 완화시킬 수 있다고 나타났다.'}

In [ ]:
orginal_text = extracted_summaries[9]['orginal_text']
summary_text = extracted_summaries[9]['summary_text']
print(orginal_text)
print(summary_text)

본 연구는 SNS의 이용과 우울의 관계에서 인지적 유연성에 의해 조절된 상향비교의 매개효과를 인스타그램을 중심으로 탐색하였다. 이를 위해 대학생 186명을 대상으로 인스타그램 이용강도, 인스타그램 이용 시 상향비교 경험, 인지적 유연성, 우울을 측정하는 척도에 응답하도록 하였다. 분석 결과 인스타그램 이용강도는 상향비교 및 우울과 정적 상관이 있었고 인지적 유연성과 부적 상관을 보였다. 전체 참가자를 대상으로 인스타그램 이용강도와 우울의 관계에서 상향비교의 매개효과를 검증한 결과 상향비교는 부분매개효과가 있는 것으로 밝혀졌으며, 매개변인인 상향비교와 종속변인인 우울의 관계를 인지적 유연성의 하위요인인 통제유연성이 조절하는 것으로 나타났다. 마지막으로 SPSS Macro를 이용하여 인스타그램 이용강도가 우울에 영향을 미침에 있어서 통제유연성에 의해 조절된 상향비교의 매개효과를 확인하였다. 이러한 결과는 인스타그램 이용강도가 강할수록 인스타그램 공간에서 상향비교를 더욱 많이 경험하게 되어 우울로 이어질 수 있지만 통제유연성을 높일 수 있다면 상향비교의 간접효과가 낮아져서 위험을 완화시킬 수 있음을 의미한다. 마지막으로 본 연구의 의의와 한계점 및 추후 연구에 대해 논의하였다.
본 연구는 인스타그램을 중심으로 SNS의 이용과 우울의 관계에 상향비교가 어떠한 매개를 하는지  탐색하였다. 분석 결과 인스타그램 이용강도가 강할수록 상향비교를 하게 되어 우울로 이어질 수 있지만 통제유연성을 높이면 상향비교를 낮출 수 있어 우울증 위험을 완화시킬 수 있다고 나타났다.


In [ ]:
llama3_summary_text = generate_response(system_message="너는 요약을 수행하는 챗봇이야. 핵심 내용만 256 토큰 이내로 한국어로 요약해줘",
                             user_message=orginal_text)
print(llama3_summary_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


한국어 요약 (256 토큰 이내):

본 연구는 인스타그램의 이용과 우울의 관계에서 인지적 유연성이 조절하는 상향비교의 매개효과를 탐색하였다. 186명을 대상으로 설문조사를 진행하여 인스타그램 이용강도, 상향비교 경험, 인지적 유연성, 우울을 측정하였다. 분석 결과 인스타그램 이용강도는 상향비교와 우울과 정적 상관을 보였으며, 인지적 유연성과 부적 상관을 보였다. 상향비교는 부분매개효과가 있는 것으로 밝혀졌으며, 통제유연성이 조절하는 것으로 나타났다. 이러한 결과는 인스타그램 이용강도가 강할수록 상향비교를 경험하게 되어 우울로 이어질 수 있지만 통제유연성을 높일 수 있다면 상향비교의 간접효과가 낮아져 위험을 완화시킬 수 있음을 의미한다.


In [ ]:
print_texts_nicely(orginal_text, summary_text, llama3_summary_text)

Original Text:
본 연구는 SNS의 이용과 우울의 관계에서 인지적 유연성에 의해 조절된 상향비교의 매개효과를 인스타그램을 중심으로 탐색하였다. 이를 위해 대학생
186명을 대상으로 인스타그램 이용강도, 인스타그램 이용 시 상향비교 경험, 인지적 유연성, 우울을 측정하는 척도에 응답하도록 하였다. 분석
결과 인스타그램 이용강도는 상향비교 및 우울과 정적 상관이 있었고 인지적 유연성과 부적 상관을 보였다. 전체 참가자를 대상으로 인스타그램
이용강도와 우울의 관계에서 상향비교의 매개효과를 검증한 결과 상향비교는 부분매개효과가 있는 것으로 밝혀졌으며, 매개변인인 상향비교와 종속변인인
우울의 관계를 인지적 유연성의 하위요인인 통제유연성이 조절하는 것으로 나타났다. 마지막으로 SPSS Macro를 이용하여 인스타그램 이용강도가
우울에 영향을 미침에 있어서 통제유연성에 의해 조절된 상향비교의 매개효과를 확인하였다. 이러한 결과는 인스타그램 이용강도가 강할수록 인스타그램
공간에서 상향비교를 더욱 많이 경험하게 되어 우울로 이어질 수 있지만 통제유연성을 높일 수 있다면 상향비교의 간접효과가 낮아져서 위험을 완화시킬
수 있음을 의미한다. 마지막으로 본 연구의 의의와 한계점 및 추후 연구에 대해 논의하였다.


Summary Text:
본 연구는 인스타그램을 중심으로 SNS의 이용과 우울의 관계에 상향비교가 어떠한 매개를 하는지  탐색하였다. 분석 결과 인스타그램 이용강도가
강할수록 상향비교를 하게 되어 우울로 이어질 수 있지만 통제유연성을 높이면 상향비교를 낮출 수 있어 우울증 위험을 완화시킬 수 있다고 나타났다.


Llama3 Summary Text:
한국어 요약 (256 토큰 이내):  본 연구는 인스타그램의 이용과 우울의 관계에서 인지적 유연성이 조절하는 상향비교의 매개효과를 탐색하였다.
186명을 대상으로 설문조사를 진행하여 인스타그램 이용강도, 상향비교 경험, 인지적 유연성, 우울을 측정하였다. 분석 결과 인스타그램 이용강도는
상향비교와 우울과 정적 상관을